In [10]:
import pyaudio
import wave
import speech_recognition as sr

FRAMES_PER_BUFFER = 3200
FORMAT            = pyaudio.paInt16 # 16-bit binary string, one bit for sign and rest for number
CHANNELS          = 1               # one source (mono)
FRAMERATE         = 16000           # no of samples per second
p = pyaudio.PyAudio()

# starts recording
stream = p.open(format=FORMAT, channels=CHANNELS, rate=FRAMERATE, input=True, frames_per_buffer=FRAMES_PER_BUFFER)
print("Start Recording ...")
frames = []
seconds = 5 # total duration of the wav file
for i in range(0, int(FRAMERATE / FRAMES_PER_BUFFER * seconds)):
    data = stream.read(FRAMES_PER_BUFFER)
    frames.append(data)
print("Recording Stopped")
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open("./files/input/output.wav", 'wb') # wb stands for write in binary mode
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(FRAMERATE)
wf.writeframes(b''.join(frames))
wf.close()

r = sr.Recognizer()
i = 0
with sr.AudioFile("./files/input/output.wav") as source:
    r.adjust_for_ambient_noise(source) # listen for 1 second to calibrate the energy threshold for ambient noise levels
    print("Converting Audio File To Text ...")
    audio = r.record(source)
    # audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        print(f"Converted Audio is: \n{text}")
        i=i+1
    except Exception as e:
        print(e)

file_text = open("./files/input/text.txt","w") # creating new file
file_text.write(text)                          # writing to text.txt file
file_text.close()                              # saving text file

if   (text == "open the door"  and i == 1):
    print("Door is Opened!")
elif (text == "close the door" and i == 2):
    print("Door is Closed!")
else: 
    print("Invalid Password. Please Try again ...")

Start Recording ...
Recording Stopped
Converting Audio File To Text ...
Converted Audio is: 
open the door
Door is Opened!
